In [1]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
import tqdm
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

In [2]:
df = pd.read_csv("C:/Users/Rebeca OdC/Documents/A4/Social media analysis/df.csv")
corpus_text = df['lead_paragraph'].values

In [3]:
df['full_text']=df['headline']+df['lead_paragraph']
corpus_text = df['full_text'].values
df=df.dropna(subset=['full_text'])
df['lenght']=df['full_text'].apply(len)

In [4]:
#CONVERTING INTO LIST OF TOKEMS AND LEMATIZATION
nlp = spacy.load("en_core_web_sm")
corpus_text=df.full_text.values
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)
docs = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]

#To avoid 'or' and 'of', I remove words that have 2 characters:
docs = [[token for token in doc if len(token) > 2] for doc in docs]

#CREATING DICTIONNARY
id2word = Dictionary(docs)
id2word.filter_extremes(no_below=20, no_above=0.1)
corpus = [id2word.doc2bow(text) for text in docs]
#testing dictionnary
print(corpus[100])

100%|██████████| 355/355 [00:09<00:00, 38.13it/s] 

[(5, 1), (14, 2), (15, 1), (25, 1), (26, 1), (30, 2), (42, 2), (43, 1), (47, 1)]


In [5]:
#Finally, the model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=15,
                   passes=20,
                   chunksize=2000)
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

#docs_2_topics will store the probability distribution of topics over documents.
docs_2_topics = lda_model[corpus]
docs_2_topics[24]

[(0, 0.17777686),
 (1, 0.011111167),
 (2, 0.17769592),
 (3, 0.011111145),
 (4, 0.01111118),
 (5, 0.011111158),
 (6, 0.011111145),
 (7, 0.011111156),
 (8, 0.011111145),
 (9, 0.0111111505),
 (10, 0.011111145),
 (11, 0.0111111505),
 (12, 0.011111145),
 (13, 0.011111162),
 (14, 0.51119334)]

In [6]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 trade, deal, one, week, sign, but, two, more, economic, other
1 :	 when, leader, economic, but, one, pandemic, take, not, company, security
2 :	 thursday, not, call, they, but, washington, take, could, two, more
3 :	 biden, foreign, tuesday, about, state, other, election, they, but, call
4 :	 state, they, secretary, company, back, pandemic, two, about, one, call
5 :	 into, election, could, washington, call, tension, one, between, after, wednesday
6 :	 about, friday, security, more, their, other, economic, monday, move, kong
7 :	 global, more, wednesday, take, use, company, could, back, foreign, security
8 :	 kong, hong, security, they, about, monday, month, tuesday, after, take
9 :	 here, your, briefing, get, sign, friday, wednesday, thursday, monday, tuesday
10 :	 between, announce, monday, two, tension, move, washington, wednesday, their, secretary
11 :	 after, people, their, day, two, about, month, take, who, friday
12 :	 pandemic, other, month, tuesday, but, global, deal, take

In [10]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.162542 -0.082352       1        1   8.905478
1      0.110215 -0.179882       2        1   5.308288
2      0.106990 -0.125504       3        1   6.605412
3     -0.067319 -0.064089       4        1   5.792724
4     -0.033033 -0.106624       5        1   7.613615
5      0.184597  0.160850       6        1   4.881427
6     -0.204297  0.065019       7        1   4.642183
7      0.057385  0.060289       8        1   6.047761
8     -0.319829 -0.000195       9        1   7.695495
9     -0.141987  0.130422      10        1   7.629548
10     0.074213  0.162349      11        1  11.596900
11    -0.063996  0.078159      12        1   7.168998
12     0.010021 -0.273647      13        1   4.661502
13     0.164552  0.196002      14        1   5.204349
14    -0.040052 -0.020798      15        1   6.246320, topic_info=         Term       Freq      Total Category  logprob  loglift
30       kong  53.000000  53.000000  Default  30.0000  30.0000
52      biden  42.000000  42.000000  Default  29.0000  29.0000
42       hong  44.000000  44.000000  Default  28.0000  28.0000
28        who  31.000000  31.000000  Default  27.0000  27.0000
35   military  28.000000  28.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
44    tension   0.062436  22.982106  Topic15  -7.3203  -3.1352
29    between   0.064791  36.669892  Topic15  -7.2833  -3.5654
25  secretary   0.062439  23.726698  Topic15  -7.3202  -3.1670
9       about   0.062771  39.957815  Topic15  -7.3150  -3.6829
13        one   0.062611  31.191635  Topic15  -7.3175  -3.4378

[611 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
9         2  0.025026  about
9         4  0.100106  about
9         5  0.100106  about
9         7  0.550581  about
9         9  0.075079  about
...     ...       ...    ...
8         2  0.874246   when
28        3  0.031589    who
28       12  0.126355    who
28       14  0.852900    who
51       10  0.971766   your

[194 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [8]:
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [9]:
#if we want to find the best article(s) for a topic
topic_number=1
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

when, leader, economic, but, one, pandemic, take, not, company, security
U.S. Small Business Bailout Money Flowed to Chinese-Owned CompaniesWASHINGTON — President Trump has blamed China for the coronavirus pandemic and the ensuing economic crisis, but as the White House looks to stabilize small businesses in the United States, the rescue effort has had an unintended beneficiary: Chinese companies. 0.8133323
How TikTok’s Owner Tried, and Failed, to Cross the U.S.-China DivideThe Chinese entrepreneur behind TikTok took ample precautions when he set out to straddle the tech world’s most treacherous divide: the one separating China’s tightly controlled internet from the rest of the planet. 0.7666658
China’s Dominance of 5G Networks Puts U.S. Economic Future at Stake, Barr WarnsWASHINGTON — Attorney General William P. Barr said Thursday that China’s dominance of 5G telecommunications networks was one of America’s top national security and economic threats, amplifying warnings issued for yea